# 1. Theoretical Framework



## 1. Objective of the index
The goal of this index is to create 

# Import Required Packages

In [72]:
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# 2. Data Selection - Importing Data

In [73]:
# This is for macroeconomic indicators
df_gdp_per_capita = pd.read_csv('GDP per capita (PPP).csv')
df_gdp_growth_rate = pd.read_csv('GDP growth rate.csv')
df_FCE = pd.read_csv('Final consumption expenditure.csv')
df_gov_debt = pd.read_excel('Government debt.xls')
df_inflation = pd.read_csv('Inflation annually.csv')

In [74]:
#This is the Trade and Investment factors
df_current_account_balance = pd.read_csv("current accoung balance (Macroeconomic Indicators ).csv")
df_exports = pd.read_csv("Exports of goods and services.csv")
df_imports = pd.read_csv("Imports of goods and services.csv")

In [75]:
# This is for Human Capital
df_labour_force =pd.read_csv('Labour Force Participant Rate.csv')
df_life_expectancy_at_birth = pd.read_csv('Life expectancy at birth.csv')
df_school_enrollment = pd.read_csv('School enrollment.csv')
df_unemployment_rate = pd.read_excel('Unemployment rate.xls')

In [76]:
#This is for infrastructure and Technology

df_access_to_electricity = pd.read_csv('Access to electricity.csv')
df_access_to_internet = pd.read_csv('indivisual using the internet.csv')
df_mobile_subscriptions = pd.read_csv('Mobile cellular subscriptions.csv')
df_LPI_Score = pd.read_excel('LPI Score.xlsx')


In [77]:
# This is for Institution and Political Factors
df_CPI = pd.read_excel("CPI2023_Global_Results__Trends.xlsx",skiprows=3, engine='openpyxl')
df_political_stability = pd.read_csv("Political Stability and Absence of ViolenceTerrorism Percentile Rank.csv")

# This is for WGI data set we will only be using the regulatory quality
df_regulatory_quality = pd.read_excel("wgidataset.xlsx")

In [78]:
#This is the environmental and sustainability

df_CO2_emissions = pd.read_csv("CO2 Emission.csv")
df_CO2_emissions_per_capita = pd.read_csv("CO2 Emission per capita.csv")
df_renewable_energy_consumption = pd.read_csv("Renewable energy consumption.csv")

# 3. Imputing of Missing Datas

In [79]:
# This is for the data cleaning process
dataframes = [df_gdp_growth_rate, df_gdp_per_capita, df_FCE, df_gov_debt, df_inflation,
              df_current_account_balance, df_exports, df_imports, df_labour_force,
              df_life_expectancy_at_birth, df_school_enrollment, df_unemployment_rate,
              df_access_to_electricity, df_access_to_internet, df_mobile_subscriptions,
              df_LPI_Score, df_CPI, df_political_stability, df_regulatory_quality,
              df_CO2_emissions, df_CO2_emissions_per_capita, df_renewable_energy_consumption]


# This is to remove the unwanted data from the dataframes
for df in dataframes:
  df.replace(["..","...", "no data"], pd.NA, inplace=True)  # or np.nan if you're using numpy


In [80]:
# Print sum of all missing valeues
def check_missing_values():
  print("The number of missing values in each dataset:")
  print("GDP per capita (PPP):", df_gdp_per_capita.isnull().sum().sum())
  print("GDP growth rate:", df_gdp_growth_rate.isnull().sum().sum())
  print("Final consumption expenditure:", df_FCE.isnull().sum().sum())
  print("Government debt:", df_gov_debt.isnull().sum().sum())
  print("Inflation annually:", df_inflation.isnull().sum().sum())
  print("Current account balance:", df_current_account_balance.isnull().sum().sum())
  print("Exports of goods and services:", df_exports.isnull().sum().sum())
  print("Imports of goods and services:", df_imports.isnull().sum().sum())
  print("Labour Force Participant Rate:", df_labour_force.isnull().sum().sum())
  print("Life expectancy at birth:", df_life_expectancy_at_birth.isnull().sum().sum())
  print("School enrollment:", df_school_enrollment.isnull().sum().sum())
  print("Unemployment rate:", df_unemployment_rate.isnull().sum().sum())
  print("Access to electricity:", df_access_to_electricity.isnull().sum().sum())
  print("Access to internet:", df_access_to_internet.isnull().sum().sum())
  print("Mobile cellular subscriptions:", df_mobile_subscriptions.isnull().sum().sum())
  print("LPI Score:", df_LPI_Score.isnull().sum().sum())
  print("CPI:", df_CPI.isnull().sum().sum())
  print("Political Stability and Absence of ViolenceTerrorism Percentile Rank:", df_political_stability.isnull().sum().sum())
  print("WGI dataset:", df_regulatory_quality.isnull().sum().sum())
  print("CO2 Emission:", df_CO2_emissions.isnull().sum().sum())
  print("CO2 Emission per capita:", df_CO2_emissions_per_capita.isnull().sum().sum())
  print("Renewable energy consumption:", df_renewable_energy_consumption.isnull().sum().sum())
  
check_missing_values()


The number of missing values in each dataset:
GDP per capita (PPP): 56
GDP growth rate: 19
Final consumption expenditure: 542
Government debt: 7
Inflation annually: 84
Current account balance: 20
Exports of goods and services: 64
Imports of goods and services: 64
Labour Force Participant Rate: 191
Life expectancy at birth: 0
School enrollment: 162
Unemployment rate: 156
Access to electricity: 9
Access to internet: 74
Mobile cellular subscriptions: 67
LPI Score: 0
CPI: 1093
Political Stability and Absence of ViolenceTerrorism Percentile Rank: 27
WGI dataset: 6756
CO2 Emission: 0
CO2 Emission per capita: 0
Renewable energy consumption: 67


In [81]:
removed_countries = []

In [82]:
removed_countries = []
country_included = None  # Set to None so we know if this is the first call

def clean_dataframe(df, country_col="Country Name"):
    global removed_countries, country_included

    # Step 1: Identify year columns
    year_columns = [col for col in df.columns if str(col).isdigit() and int(col) <= 2023]
    year_columns = sorted(year_columns, reverse=True)

    # Step 2: Convert year columns to numeric
    for col in year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Step 3: Restrict to previously included countries, if any
    if country_included is not None:
        df = df[df[country_col].isin(country_included)].copy()

    # Step 4: Identify and record countries to remove
    missing_counts = df[year_columns].isna().sum(axis=1)
    to_remove = df[missing_counts >= 5]
    removed_countries += to_remove[country_col].dropna().tolist()

    # Step 5: Drop rows from removed countries
    df = df[~df[country_col].isin(removed_countries)].copy()

    # Step 6: Update country_included if it's the first dataset
    if country_included is None:
        country_included = df[country_col].dropna().unique().tolist()

    # Step 7: Fill missing year values with row mean
    def fill_row_mean(row):
        row[year_columns] = row[year_columns].fillna(row[year_columns].mean())
        return row

    df = df.apply(fill_row_mean, axis=1)
    
    return df


In [83]:
df_gdp_per_capita = clean_dataframe(df_gdp_per_capita)
df_gdp_growth_rate = clean_dataframe(df_gdp_growth_rate)
df_FCE = clean_dataframe(df_FCE)
df_gov_debt = clean_dataframe(df_gov_debt)
df_inflation = clean_dataframe(df_inflation)


df_gdp_per_capita = df_gdp_per_capita[['Country Name', "2023"]].rename(columns={"2023": 'gdp_per_capita_2023'})
df_gdp_growth_rate = df_gdp_growth_rate[['Country Name', "2023"]].rename(columns={"2023": 'gdp_growth_rate_2023'})
df_FCE = df_FCE[['Country Name', "2023"]].rename(columns={"2023": 'final_consumption_expenditure_2023'})

df_gov_debt = df_gov_debt[['Country Name', 2023]].rename(columns={2023: 'government_debt_2023'})
df_inflation = df_inflation[['Country Name', "2023"]].rename(columns={"2023": 'inflation_rate_2023'})


In [84]:
df_current_account_balance = clean_dataframe(df_current_account_balance)
df_exports = clean_dataframe(df_exports)
df_imports = clean_dataframe(df_imports)

df_current_account_balance = df_current_account_balance[['Country Name', "2023"]].rename(columns={"2023": 'current_account_balance_2023'})
df_exports = df_exports[['Country Name', "2023"]].rename(columns={"2023": 'exports_2023'})
df_imports = df_imports[['Country Name', "2023"]].rename(columns={"2023": 'imports_2023'})

In [85]:
df_labour_force = clean_dataframe(df_labour_force)
df_life_expectancy_at_birth = clean_dataframe(df_life_expectancy_at_birth)
df_school_enrollment = clean_dataframe(df_school_enrollment)
df_unemployment_rate = clean_dataframe(df_unemployment_rate)

df_labour_force = df_labour_force[['Country Name', "2023"]].rename(columns={"2023": 'labour_force_participation_rate_2023'})
df_life_expectancy_at_birth = df_life_expectancy_at_birth[['Country Name', "2023"]].rename(columns={"2023": 'life_expectancy_at_birth_2023'})
df_school_enrollment = df_school_enrollment[['Country Name', "2023"]].rename(columns={"2023": 'school_enrollment_rate_2023'})
df_unemployment_rate = df_unemployment_rate[['Country Name', 2023]].rename(columns={2023: 'unemployment_rate_2023'})

In [86]:
df_access_to_electricity = clean_dataframe(df_access_to_electricity)
df_access_to_internet = clean_dataframe(df_access_to_internet)
df_mobile_subscriptions = clean_dataframe(df_mobile_subscriptions)

df_access_to_electricity = df_access_to_electricity[['Country Name', "2023"]].rename(columns={"2023": 'access_to_electricity_2023'})
df_access_to_internet = df_access_to_internet[['Country Name', "2023"]].rename(columns={"2023": 'access_to_internet_2023'})
df_mobile_subscriptions = df_mobile_subscriptions[['Country Name', "2023"]].rename(columns={"2023": 'mobile_subscriptions_2023'})

In [87]:
df_LPI_Score = df_LPI_Score[["Economy", "LPI Score"]]
df_LPI_Score.rename(columns={
    "Economy": "Country Name",
    "LPI Score": "lpi_score_2023"
}, inplace=True)


C:\Users\DEcla\AppData\Local\Temp\ipykernel_36416\2614403127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LPI_Score.rename(columns={


In [88]:
df_CPI = df_CPI[df_CPI['Country / Territory'].isin(country_included)][['Country / Territory', 'CPI score 2023']]
df_CPI.rename(columns={
    "Country / Territory": "Country Name",
    "CPI score 2023": "cpi_score_2023"
}, inplace=True)

df_political_stability  = clean_dataframe(df_political_stability, country_col="Country Name")
df_political_stability = df_political_stability[['Country Name', "2023"]].rename(columns={"2023": 'political_stability_2023'})

df_regulatory_quality = df_regulatory_quality[
    (df_regulatory_quality['indicator'] == 'rq') &
    (df_regulatory_quality['year'] > 2014) & 
    (df_regulatory_quality['year'] < 2023) & df_regulatory_quality['countryname'].isin(country_included) 
]

df_regulatory_quality = (
    df_regulatory_quality
    .groupby('countryname')['estimate']
    .mean()
    .reset_index()
    .rename(columns={'estimate': 'regulatory_quality_2023', 'countryname': 'Country Name'})
)




In [89]:
df_CO2_emissions = clean_dataframe(df_CO2_emissions)
df_CO2_emissions_per_capita = clean_dataframe(df_CO2_emissions_per_capita)
df_renewable_energy_consumption = clean_dataframe(df_renewable_energy_consumption)

df_CO2_emissions = df_CO2_emissions[['Country Name', "2023"]].rename(columns={"2023": 'co2_emissions_2023'})
df_CO2_emissions_per_capita = df_CO2_emissions_per_capita[['Country Name', "2023"]].rename(columns={"2023": 'co2_emissions_per_capita_2023'})
df_renewable_energy_consumption = df_renewable_energy_consumption[['Country Name', "2023"]].rename(columns={"2023": 'renewable_energy_consumption_2023'})

In [90]:
check_missing_values()

The number of missing values in each dataset:
GDP per capita (PPP): 0
GDP growth rate: 0
Final consumption expenditure: 0
Government debt: 0
Inflation annually: 0
Current account balance: 0
Exports of goods and services: 0
Imports of goods and services: 0
Labour Force Participant Rate: 0
Life expectancy at birth: 0
School enrollment: 0
Unemployment rate: 0
Access to electricity: 0
Access to internet: 0
Mobile cellular subscriptions: 0
LPI Score: 0
CPI: 0
Political Stability and Absence of ViolenceTerrorism Percentile Rank: 0
WGI dataset: 0
CO2 Emission: 0
CO2 Emission per capita: 0
Renewable energy consumption: 0


In [91]:

from functools import reduce
dfs = [df_gdp_per_capita, df_gdp_growth_rate, df_FCE, df_gov_debt, df_inflation]
df_combined = reduce(lambda left, right: pd.merge(left, right, on='Country Name', how='inner'), dfs)

print(df_combined.head())


  Country Name  gdp_per_capita_2023  gdp_growth_rate_2023  \
0    Australia          70497.15245              3.441992   
1    Australia          70497.15245              3.441992   
2    Australia          70497.15245              3.441992   
3    Australia          70497.15245              3.441992   
4    Australia          70497.15245              3.441992   

   final_consumption_expenditure_2023  government_debt_2023  \
0                           71.652646             34.832823   
1                           21.313855             34.832823   
2                           50.338791             34.832823   
3                           23.974017             34.832823   
4                           26.799241             34.832823   

   inflation_rate_2023  
0             5.597015  
1             5.597015  
2             5.597015  
3             5.597015  
4             5.597015  
